In [3]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

data_path = r'C:\Users\Subways-Sun\OneDrive\Documents\GitHub\sodium-ion-batteries\data_annotated\annotated_data_openai.json'
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

texts = data["text"]
labels = data["label_annotated"]

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, 
    labels, 
    test_size=0.2, 
    random_state=42
)

train_dataset = Dataset.from_dict({
    'text': train_texts,
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'text': val_texts,
    'labels': val_labels
})

raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [4]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 120
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 30
    })
})


In [6]:
model_name = "batterydata/batterybert-cased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  eval_strategy="epoch",
                                  num_train_epochs=7)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)


In [18]:
trainer.train()

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.351445734500885, 'eval_accuracy': 0.9333333333333333, 'eval_runtime': 0.3063, 'eval_samples_per_second': 97.943, 'eval_steps_per_second': 13.059, 'epoch': 1.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.422661691904068, 'eval_accuracy': 0.9333333333333333, 'eval_runtime': 0.2669, 'eval_samples_per_second': 112.392, 'eval_steps_per_second': 14.986, 'epoch': 2.0}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.2854878306388855, 'eval_accuracy': 0.9666666666666667, 'eval_runtime': 0.2852, 'eval_samples_per_second': 105.181, 'eval_steps_per_second': 14.024, 'epoch': 6.0}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.28339827060699463, 'eval_accuracy': 0.9666666666666667, 'eval_runtime': 0.2738, 'eval_samples_per_second': 109.574, 'eval_steps_per_second': 14.61, 'epoch': 7.0}
{'train_runtime': 26.9676, 'train_samples_per_second': 31.148, 'train_steps_per_second': 3.894, 'train_loss': 0.1320663815452939, 'epoch': 7.0}


TrainOutput(global_step=105, training_loss=0.1320663815452939, metrics={'train_runtime': 26.9676, 'train_samples_per_second': 31.148, 'train_steps_per_second': 3.894, 'total_flos': 221013286502400.0, 'train_loss': 0.1320663815452939, 'epoch': 7.0})